<a href="https://colab.research.google.com/github/MahiLak-AI/AI-ML-/blob/main/IMDBreviewsetimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install kaggle

Imporiting the dependencies

In [23]:
import os
import json
from zipfile import ZipFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection from KAGGLE  api

```
# This is formatted as code
```



In [24]:
kaggle_dictionary=json.load(open("kaggle.json"))
#Setting up kaggle credentials in the environment
os.environ["KAGGLE_USERNAME"]=kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"]=kaggle_dictionary["key"]



In [25]:
#Downloading Data from kaggle
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


Unzipping the Dataset file

In [26]:
with ZipFile("/content/imdb-dataset-of-50k-movie-reviews.zip","r") as zip:
  zip.extractall()

In [27]:
!ls

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


Loading the Dataset and Preprocessing Data

In [28]:
df=pd.read_csv("/content/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [29]:
df.shape

(50000, 2)

Data has 50000 rows and 2 columns

In [30]:
df["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [31]:
df.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-31-fb62f5dd8c08>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


#Split data into train and test

In [32]:
train_data,test_data=train_test_split(df,test_size=0.2,random_state=42)
train_data.shape,test_data.shape

((40000, 2), (10000, 2))

#Data Preprocessing


In [33]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [34]:
y_train=train_data["sentiment"]
y_test=test_data["sentiment"]


## New Section

#LSTM Model

In [35]:
 #build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [36]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 460s 366ms/step - accuracy: 0.7369 - loss: 0.5150 - val_accuracy: 0.8631 - val_loss: 0.3329
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 538s 394ms/step - accuracy: 0.8613 - loss: 0.3357 - val_accuracy: 0.8735 - val_loss: 0.3037
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 463s 363ms/step - accuracy: 0.8991 - loss: 0.2535 - val_accuracy: 0.8905 - val_loss: 0.2798
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 506s 367ms/step - accuracy: 0.9149 - loss: 0.2192 - val_accuracy: 0.8807 - val_loss: 0.3013
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 457s 366ms/step - accuracy: 0.9320 - loss: 0.1770 - val_accuracy: 0.8914 - val_loss: 0.2877
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 503s 367ms/step - accuracy: 0.9427 - loss: 0.1535 - val_accuracy: 0.8910 - val_loss: 0.2845
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 503s 368ms/step - accuracy: 0.9529 - loss: 0.1283 - val_accuracy: 0.8932 - val_loss: 0.3042
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 498s 365ms/step - ac

In [39]:
loss,accuracy=model.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 33s 106ms/step - accuracy: 0.8856 - loss: 0.3908


#Building a predictive system

In [ ]:
def predict_sentiment(review):
  #Tokenize and pad the review first
  sequence=tokenizer.text_to_sequence([review])
  pad_sequence=pad_sequences(sequence,maxlen=200)
  #Make prediction
  prediction=model.predict(pad_sequence)
  sentiment="positive" if prediction[0][0]>0.5 else "negative"
  return sentiment

In [ ]:
#Example
review="The movie was magnifficent. I enjoyed watching the movie"
sentiment = predict_sentiment(review)
print("The movie sentiment is ",sentiment)